### Parte I - Modelo de programación lineal y metaheurística de Ant Colony

In [ ]:
from docplex.mp.model import Model
import random
import numpy as np

#### Parámetros del problema

In [ ]:
# Conjuntos
origenes = ['A', 'B', ???]  # A: Entre Ríos, B: Catamarca, C: Comodoro Rivadavia
destinos = ['X', 'Y']       # X: Neuquén, Y: Santa Cruz

# Costos base por tonelada (USD)
costo_arena = {
    'A': 2000,  # Entre Ríos
    'B': 2500,  # Catamarca
    'C': 2800   # Importada vía Comodoro
}

# Distancias (km)
distancia = {
    ('A', 'X'): 1500,
    ('A', 'Y'): 2000,
    ('B', 'X'): 500,
    ('B', 'Y'): 1000,
    ('C', 'X'): 450,
    ('C', 'Y'): 250
}

# Costo total por tonelada = costo base + distancia
costo_total = {(i, j): costo_arena[i] + distancia[i, j] for i in origenes for j in ???}

# Rutas posibles
rutas = [('A', 'X'), ('A', 'Y'), ('B', 'X'), ('B', 'Y'), ('C', 'X'), ('C', 'Y')]

# Parámetros del problema
demanda = {'X': 7, 'Y': 3}
max_camiones = 10
comodoro_limit = {'C_X': 1, 'C_Y': 1}


print(costo_total)

#### Solución via PL

In [ ]:
# Crear modelo
mdl = Model("Asignacion_Camiones_Distancia")

# Variables: x_ij = cantidad de camiones asignados (toneladas) de i a j
x = mdl.integer_var_dict(((i, j) for i in origenes for j in destinos), name='x')

# Restricción de demanda
mdl.add_constraint(x['A', 'X'] + x['B', 'X'] + x['C', 'X'] == demanda['X'], "demanda_Neuquen")
mdl.add_constraint(x['A', 'Y'] + x['B', 'Y'] + x['C', 'Y'] == demanda['Y'], "demanda_SantaCruz")

# Límite de camiones totales
mdl.add_constraint(mdl.sum(x[i, j] for i in origenes for j in destinos) <= 10, "camiones_totales")

# Restricciones de capacidad por ruta desde Comodoro
mdl.add_constraint(x['C', 'X'] <= ???
mdl.add_constraint(x['C', 'Y'] <= ???

# Objetivo: minimizar costo total
mdl.minimize(mdl.sum(costo_total[i, j] * x[i, j] for i in origenes for j in destinos))

# Resolver
sol = mdl.solve(log_output=True)

# Resultados
if sol:
    print("\nSolución óptima encontrada:\n")
    for i in origenes:
        for j in destinos:
            val = x[i, j].solution_value
            if val > 0:
                print(f"Camiones desde {i} hacia {j}: {int(val)}")
    print(f"\nCosto total mínimo: ${mdl.objective_value:.2f}")
else:
    print("No se encontró una solución.")


#### Solución vía Ant Colony

In [ ]:
# Parámetros metaheurística Ant Colony
num_hormigas = ??
num_iter = ??
evap = ??
alpha = 1.0
beta = 2.0

# Inicializar feromonas
feromona = {r: 1.0 for ? in ?}
mejor_costo = float('inf')
mejor_solucion = None

for iteracion in range(???):
    soluciones = []
    costos_por_solucion = []

    for ant in range(??):
        x = {r: 0 for r in rutas}
        total_camiones = 0
        carga = {'X': 0, 'Y': 0}

        rutas_disponibles = rutas.copy()
        while rutas_disponibles and total_camiones < max_camiones:
            # Elegir una ruta con probabilidad proporcional a feromona / costo
            scores = []
            for r in rutas_disponibles:
                i, j = r
                eta = 1 / costo_total[r]
                tau = feromona[r]
                scores.append((tau ** alpha) * (eta ** beta))

            probs = np.array(scores) / sum(scores)
            idx = np.random.choice(len(rutas_disponibles), p=probs)
            ruta = rutas_disponibles[idx]
            i, j = ruta

            # Validar restricciones antes de asignar
            if carga[j] < demanda[j]:
                # Restricciones de Comodoro
                if i == 'C' and x[ruta] >= 1:
                    rutas_disponibles.remove(ruta)
                    continue

                x[ruta] += 1
                carga[j] += 1
                total_camiones += 1

            # Si ya cumplió demanda, quitar ruta
            if carga[j] >= ???:
                rutas_disponibles = [r for r in rutas_disponibles if r[1] != j]

        # Validar solución
        if carga == demanda and total_camiones <= max_camiones:
            costo_actual = sum(x[r] * costo_total[r] for r in rutas)
            soluciones.append(x.copy())
            costos_por_solucion.append(costo_actual)

            if costo_actual < ???:
                mejor_costo = costo_actual
                mejor_solucion = x.copy()

    # Evaporación
    for r in rutas:
        feromona[r] *= (1 - evap)

    # Refuerzo con mejores soluciones
    if soluciones:
        mejor_indice = np.argmin(costos_por_solucion)
        for r in rutas:
            feromona[r] += 1.0 / costos_por_solucion[mejor_indice] * soluciones[mejor_indice][r]

#print(mejor_solucion)
print("\nMejor solución encontrada:")
for r in rutas:
    if mejor_solucion[r] > 0:
        print(f"Camiones desde {r[0]} hacia {r[1]}: {mejor_solucion[r]}")
print(f"Costo total: ${mejor_costo:.2f}")


### Parte II - Sensibilidad de solución

##### b. ¿A partir de que costo de la arena importada esta deja de estar en la solución?